# Jacques Pyrometer Testing

In [1]:
#!/usr/bin/env python3
import minimalmodbus
import serial

instrument = minimalmodbus.Instrument('COM3', 1, debug = True)  # port name, slave address (in decimal)

instrument.serial.port = 'COM3'                     # this is the serial port name
instrument.serial.baudrate = 115200         # Baud
# instrument.serial.baudrate = 950         # Baud
instrument.serial.bytesize = 8
instrument.serial.parity   = serial.PARITY_NONE
instrument.serial.stopbits = 1
instrument.serial.timeout  = 0.05          # seconds

instrument.mode = minimalmodbus.MODE_RTU   # rtu or ascii mode
instrument.clear_buffers_before_each_transaction = True
# print(instrument.read_register(289, 1))
## Read temperature (PV = ProcessValue) ##
temperature = instrument.read_register(3, 3)  # Registernumber, number of decimals
# print(temperature)
# 01 03 00 01 00 01 D5 CA (8 bytes)
# 01 03 00 02 00 01 25 CA
# 01 03 00 03 00 01 74 0A


MinimalModbus debug mode. Create serial port COM3
MinimalModbus debug mode. Will write to instrument (expecting 7 bytes back): 01 03 00 03 00 01 74 0A (8 bytes)
MinimalModbus debug mode. Clearing serial buffers for port COM3
MinimalModbus debug mode. No sleep required before write. Time since previous read: 302520421.00 ms, minimum silent period: 1.75 ms.
MinimalModbus debug mode. Response from instrument:  (0 bytes), roundtrip time: 0.0 ms. Timeout for reading: 50.0 ms.



NoResponseError: No communication with the instrument (no answer)

In [ ]:
import serial
import time

# Open COM4 at 9600 baud, 8N1 (no parity, 1 stop bit)
ser = serial.Serial(
    port="COM3",
    baudrate=115200,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1  # 1 second read timeout
)

def send_command(command_bytes, expect_response=True):
    """Send a list of bytes and optionally read back a response."""
    ser.write(bytes(command_bytes))
    print(f"Sent: {[hex(b) for b in command_bytes]}")

    if expect_response:
        time.sleep(0.1)  # wait a little for device to respond
        response = ser.read(100)  # read up to 100 bytes
        print("Received:", response.hex(" ").upper())
        return response
    else:
        print("No response expected.")
        return None

# --- Example 1: Switch to Modbus Mode (no response expected)
send_command([0x02, 0x4D, 0x4D, 0x03])

# --- Example 2: Report Version (expects a response)
send_command([0x02, 0x56, 0x56, 0x03])
# 02 31 31 03
send_command([0x02, 0x31, 0x31, 0x03])
ser.close()


Sent: ['0x2', '0x4d', '0x4d', '0x3']
Received: 02 4D 4D 03
Sent: ['0x2', '0x56', '0x56', '0x3']
Received: 02 56 56 03
Sent: ['0x2', '0x31', '0x31', '0x3']
Received: 02 31 31 03


In [4]:
import serial
import time

ser = serial.Serial(
    port='COM3',
    baudrate=115200,
    bytesize=8,
    parity='N',
    stopbits=1,
    timeout=0.1
)

cmd = bytes([0x02, 0x31, 0x31, 0x03])
ser.write(cmd)
print("Sent:", ' '.join(f'{b:02X}' for b in cmd))

print("Reading for 5 seconds...")
end_time = time.time() + 5
while time.time() < end_time:
    response = ser.read(100)
    if response:
        print("Received:", ' '.join(f'{b:02X}' for b in response))

ser.close()


Sent: 02 31 31 03
Reading for 5 seconds...
Received: 02 31 31 03
